In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import io
import itertools
import sklearn.metrics
import datetime
import matplotlib.pyplot as plt

In [3]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [4]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [5]:
no_of_validation_samples=0.1*mnist_info.splits['train'].num_examples
no_of_validation_samples=tf.cast(no_of_validation_samples,tf.int64)

no_of_test_samples=mnist_info.splits['test'].num_examples
no_of_test_samples=tf.cast(no_of_test_samples,tf.int64)

In [6]:
buffer_size=70000
batch_size=128
max_epoch=20

In [7]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

In [8]:
scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test_data=mnist_test.map(scale)

In [9]:
suffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

In [10]:
validation_data=suffled_train_and_validation_data.take(no_of_validation_samples)
train_data=suffled_train_and_validation_data.skip(no_of_validation_samples)

In [11]:
train_data=train_data.batch(batch_size)
validation_data=validation_data.batch(no_of_validation_samples)
test_data=scaled_test_data.batch(no_of_test_samples)

In [12]:
for images,labels in validation_data:
    images_val=images.numpy()
    labels_val=images.numpy()

In [13]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,5,activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50,3,activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [14]:
loss_rn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits='softmax')
model.compile(optimizer='adam',loss=loss_rn,metrics=['accuracy'])

In [15]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto',min_delta=0,verbose=0,patience=2,restore_best_weights=True)

In [16]:
log_dir="logs\\fit\\"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [17]:
model.fit(train_data,epochs=max_epoch,callbacks=[tensorboard_callback,early_stopping],validation_data=validation_data,verbose=2)

Epoch 1/20


422/422 - 173s - loss: 0.2599 - accuracy: 0.9304 - val_loss: 0.0810 - val_accuracy: 0.9757
Epoch 2/20
422/422 - 147s - loss: 0.0703 - accuracy: 0.9790 - val_loss: 0.0505 - val_accuracy: 0.9847
Epoch 3/20
422/422 - 153s - loss: 0.0511 - accuracy: 0.9848 - val_loss: 0.0500 - val_accuracy: 0.9862
Epoch 4/20
422/422 - 130s - loss: 0.0418 - accuracy: 0.9871 - val_loss: 0.0326 - val_accuracy: 0.9900
Epoch 5/20
422/422 - 128s - loss: 0.0379 - accuracy: 0.9878 - val_loss: 0.0356 - val_accuracy: 0.9897
Epoch 6/20
422/422 - 152s - loss: 0.0324 - accuracy: 0.9900 - val_loss: 0.0271 - val_accuracy: 0.9910
Epoch 7/20
422/422 - 119s - loss: 0.0283 - accuracy: 0.9912 - val_loss: 0.0187 - val_accuracy: 0.9948
Epoch 8/20
422/422 - 138s - loss: 0.0242 - accuracy: 0.9929 - val_loss: 0.0144 - val_accuracy: 0.9947
Epoch 9/20
422/422 - 160s - loss: 0.0216 - accuracy: 0.9931 - val_loss: 0.0124 - val_accuracy: 0.9960
Epoch 10/20
422/422 - 204s - loss: 0.0191 - accuracy: 0.9939 - val_loss: 0.0143 - val_accurac

In [18]:
%load_ext tensorboard
%tensorboard --logdir 'logs/fit'

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 12224.

In [19]:
test_loss,test_accuracy=model.evaluate(test_data)

      1/Unknown - 55s 55s/step - loss: 0.0299 - accuracy: 0.9907